<a href="https://colab.research.google.com/github/Nik8x/TF_Speech_Recognition/blob/master/TF_Speech_Recognition_test_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import glob
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
import IPython

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Conv2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

import os

# Any results you write to the current directory are saved as output.
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
print("The current directory is: ", os.getcwd())
import os
os.chdir("/content/gdrive/My Drive/Galvanize Adm/TF_Speech_Recognition")
print("The current directory is: ", os.getcwd())

The current directory is:  /content
The current directory is:  /content/gdrive/My Drive/Galvanize Adm/TF_Speech_Recognition


In [0]:
os.listdir()

['test_list.txt',
 'val_list.txt',
 'dfff.csv',
 'array.csv',
 'sample_submission.7z',
 'train.7z',
 'test.7z',
 'test_1',
 'test_df.csv',
 'tfsr.h5',
 'sample_submission.csv',
 'TF_Speech_Recognition.ipynb']

In [0]:
#!sudo apt install p7zip

In [0]:
#!p7zip -d test.7z

In [0]:
# # let's unzip test.7z 
# from zipfile import ZipFile
# zf = ZipFile('test.7z', 'r')
# zf.extractall('.')
# zf.close()

In [0]:
# for use in Kaggle as the audio files are not saved in Colabs - Loading the files, let's see total audio files for each label
# label = []
# counts = []
# for i in os.listdir('../input/train/audio/'):
#     label.append(i)
#     counts.append(len(os.listdir('../input/train/audio/' + i)))
    
# df = pd.DataFrame([label, counts]).T.set_index([0])
# df

In [0]:
# # Total number of audio files
# np.sum(df[1])

In [0]:
# os.listdir('../input/')

In [0]:
# Saving files from test and validation list
# test_list = open('../input/train/testing_list.txt').read().split('\n')
# val_list = open('../input/train/validation_list.txt').read().split('\n')

In [0]:
# # let us save the above list in our directory, so we can load it afterwards.  This will be easier for us to avoid opening the aduio files
# with open("test_list.txt", "w") as output:
#     output.write(str(test_list))
    
# with open("val_list.txt", "w") as output:
#     output.write(str(val_list))

In [0]:
# Checking the lenght of the list
# len(test_list), len(val_list)

In [0]:
#Playint on audio file
# IPython.display.Audio('../input/train/audio/right/bf8d5617_nohash_0.wav')

In [0]:
#Labels that Kaggle has asked us to use for the competition
# labels_in_test = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
# df = df.loc[labels_in_test]
# df

In [0]:
# Total number of files in the labels_in_test
# np.sum(df[1])

In [0]:
# Loading the audio file as an array using the librosa's mfcc - we do this since our keras model only understands arrays(or numbers) and we
# cannot feed the audio file directly.
# %%time

# def namestr(obj, namespace):
#     return [name for name in namespace if namespace[name] is obj]

# def extract_data(folder):
#     dff = pd.DataFrame()
#     names = []
#     features = []
#     data = []
#     sampling_rate = []
    
#     for name in os.listdir('../input/train/audio/' + folder):
#         data1, sampling_rate1 = librosa.load('../input/train/audio/' + folder + '/' + name, res_type = 'kaiser_fast')
#         mfccs = np.mean(librosa.feature.mfcc(y = data1, sr = sampling_rate1, n_mfcc = 40).T, axis = 0)
        
#         data.append(data1)
#         sampling_rate.append(sampling_rate1)
#         features.append(mfccs)
#         names.append(name)
        
#     return [names, data, sampling_rate, features]

# train_new = pd.Series(labels_in_test).apply(extract_data)
# train_new.columns = ['names', 'data', 'sampling_rate', 'features']

In [0]:
# Connecting all the lists into one DataFrame
# dfff = pd.DataFrame()
# dfff['names'] = train_new[0][0] + train_new[1][0] + train_new[2][0] + train_new[3][0] + train_new[4][0] + train_new[5][0] + train_new[6][0] + train_new[7][0] + train_new[8][0] + train_new[9][0]
# dfff['data'] = train_new[0][1] + train_new[1][1] + train_new[2][1] + train_new[3][1] + train_new[4][1] + train_new[5][1] + train_new[6][1] + train_new[7][1] + train_new[8][1] + train_new[9][1]
# dfff['sampling_rate'] = train_new[0][2] + train_new[1][2] + train_new[2][2] + train_new[3][2] + train_new[4][2] + train_new[5][2] + train_new[6][2] + train_new[7][2] + train_new[8][2] + train_new[9][2]
# dfff['features'] = train_new[0][3] + train_new[1][3] + train_new[2][3] + train_new[3][3] + train_new[4][3] + train_new[5][3] + train_new[6][3] + train_new[7][3] + train_new[8][3] + train_new[9][3]

In [0]:
# Making a new list for the label column
# label_all = []
# for index in labels_in_test:
#     label_all.append([index] * df.loc[index][1])

In [0]:
# new list for the label column is now being put into the DataFrame
# dfff['label'] = [item for sublist in label_all for item in sublist]

In [0]:
# print(dfff.shape)
# dfff.head()

In [0]:
# Saving the DataFrame as a csv file to avoid using the audio files going forward
# dfff.to_csv("dfff.csv", index = False, sep = ',')

In [0]:
# Converting the string of arrays to a proper array
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')

In [0]:
dfff = pd.read_csv("dfff.csv", converters = {'features' : converter})

In [0]:
dfff.head()

,Unnamed: 0,names,feature,label
0,0,5fe4a278_nohash_1.wav,"[-360.960392, 95.2412528, -45.965287, 61.02619...",yes
1,1,4def68db_nohash_1.wav,"[-350.560308, 95.8547696, -18.2142555, 43.4533...",yes
2,2,3d53244b_nohash_3.wav,"[-452.935767, 48.5914362, -8.35673584, 39.4135...",yes
3,3,62ef962d_nohash_1.wav,"[-304.877085, 60.8177152, -32.5946494, 18.1406...",yes
4,4,264f471d_nohash_3.wav,"[-305.40941638, 100.41015856, -2.31436065, 58....",yes


In [0]:
# Loading the test and validation list
import ast
with open("test_list.txt", "r") as f:
    test_list = ast.literal_eval(f.read())
    
with open("val_list.txt", "r") as f:
    val_list = ast.literal_eval(f.read())

In [0]:
# Extracting the names of the files which Kaggle has asked us to validate upon
val_list_all = pd.Series(val_list[:-1]).apply(lambda x: x.split('/')[1])
test_list_all = pd.Series(test_list[:-1]).apply(lambda x: x.split('/')[1])
len(val_list_all), len(test_list_all)

(6798, 6835)

In [0]:
# Making a DataFrame from the val_list
val = dfff[dfff.names.isin(val_list_all)]
val = val.iloc[np.random.permutation(len(val))]  # Shuffle DataFrame rows
print(val.shape)

train = dfff[~dfff.names.isin(val_list_all)]
train = train.iloc[np.random.permutation(len(train))] # Shuffle DataFrame rows
print(train.shape)

(2577, 4)
(21105, 4)


In [0]:
train.head()

,Unnamed: 0,names,feature,label
8754,8754,3402e488_nohash_2.wav,"[-549.733726, 79.4955328, -30.9182076, 26.1704...",down
7337,7337,13d7b8c9_nohash_0.wav,"[-344.692373, 90.9073308, -45.5252204, 17.4863...",down
9957,9957,11b1df78_nohash_1.wav,"[-364.480762, 185.151613, -12.7927503, 26.2872...",left
21163,21163,39a12648_nohash_2.wav,"[-211.56315848, 99.26927032, -70.97281211, 47....",stop
21926,21926,3e31dffe_nohash_1.wav,"[-480.62956324, 129.7252135, -0.73859652, 21.9...",go


In [0]:
#Splitting into our X and y array
X_train = np.array(train.features.tolist())
y_train = np.array(train.label.tolist())

X_val = np.array(val.features.tolist())
y_val = np.array(val.label.tolist())

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_val = np_utils.to_categorical(lb.fit_transform(y_val))

In [0]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((21105, 40), (21105, 10), (2577, 40), (2577, 10))

In [0]:
# # Here we are using deep learning using a sequential model with three hidden layers
# ncols = X_train.shape[1]
# model = Sequential()

# model.add(Dense(1024, activation = 'relu', input_shape = (ncols,)))
# model.add(Dropout(0.5))

# model.add(Dense(512, activation = 'relu'))
# model.add(Dropout(0.5))

# model.add(Dense(256, activation = 'relu'))
# model.add(Dropout(0.5))

# model.add(Dense(128, activation = 'relu'))
# model.add(Dropout(0.5))

# model.add(Dense(y_train.shape[1], activation = 'softmax'))

# model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
# # Fitting the model
# %%time
# model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = [X_val, y_val])

In [0]:
# # Applying CNN (deep learning)
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(8, 5, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(y_train.shape[1], activation = 'softmax'))

# model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# model.summary()

# Applying CNN (deep learning)
# %%time
# model.fit(X_train.reshape(X_train.shape[0], 8, 5, 1), y_train, batch_size = 32, epochs = 50, validation_data = [X_val.reshape(X_val.shape[0], 8, 5, 1), y_val], verbose = 1)

In [0]:
# Applying CNN (deep learning) with tpu
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(8, 5, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(y_train.shape[1], activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 4, 32)          160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 2, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 2, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 192)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               24704     
_________________________________________________________________
dropout_1 (Dropout)  

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.86.123.98:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 9908395419379460952)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11439517981440160975)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1212404706152780327)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1

In [0]:
tpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    (None, 8, 5, 1)           0         
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 4, 32)          160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 2, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 2, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 192)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               24704     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [0]:
# Applying CNN (deep learning)
%%time
tpu_model.fit(X_train.reshape(X_train.shape[0], 8, 5, 1), y_train, batch_size = 32, epochs = 150, validation_data = [X_val.reshape(X_val.shape[0], 8, 5, 1), y_val], verbose = 1)

Train on 21105 samples, validate on 2577 samples
Epoch 1/150
21105/21105 [==============================] - 8s 364us/sample - loss: 1.7841 - acc: 0.3632 - val_loss: 1.6794 - val_acc: 0.4301
Epoch 2/150
21105/21105 [==============================] - 8s 372us/sample - loss: 1.7863 - acc: 0.3624 - val_loss: 1.6762 - val_acc: 0.4293
Epoch 3/150
21105/21105 [==============================] - 8s 361us/sample - loss: 1.7802 - acc: 0.3651 - val_loss: 1.6734 - val_acc: 0.4262
Epoch 4/150
21105/21105 [==============================] - 8s 363us/sample - loss: 1.7749 - acc: 0.3632 - val_loss: 1.6626 - val_acc: 0.4328
Epoch 5/150
21105/21105 [==============================] - 8s 364us/sample - loss: 1.7757 - acc: 0.3651 - val_loss: 1.7054 - val_acc: 0.4243
Epoch 6/150
21105/21105 [==============================] - 8s 359us/sample - loss: 1.7836 - acc: 0.3617 - val_loss: 1.6780 - val_acc: 0.4266
Epoch 7/150
21105/21105 [==============================] - 7s 352us/sample - loss: 1.7891 - acc: 0.3587 -

In [0]:
# # Applying CNN with 4 hidden layers, out of which 1 is a CNN layer and 3 are dense layers (deep learning)
# model = Sequential()
# model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', input_shape=(8, 5, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(32, (2, 2), padding="same", activation="relu", input_shape=(8, 5, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())

# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.25))
          
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.25))
          
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.25))

# model.add(Dense(y_train.shape[1], activation = 'softmax'))

# model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# model.summary()

# # Fitting the model 
# %%time
# model.fit(X_train.reshape(X_train.shape[0], 8, 5, 1), y_train, batch_size = 32, epochs = 50, validation_data = [X_val.reshape(X_val.shape[0], 8, 5, 1), y_val], verbose = 1)

In [0]:
model.save_weights('tfsr.h5')

In [0]:
model.load_weights('tfsr.h5')

In [0]:
test_df = pd.read_csv('test_df.csv', converters = {'features' : converter, 'data' : converter})
test_df.head()

,names,data,sampling_rate,features
0,clip_000044442.wav,"[0.00213205, 0.00348907, 0.00481788]",22050,"[-343.180481, 108.435396, -8.90255396, 47.3082..."
1,clip_0000adecb.wav,"[-0.00020656, 0.00100615, 0.0023665]",22050,"[-537.140085, 85.9404343, -17.808872, 34.89341..."
2,clip_0000d4322.wav,"[-5.3789678e-05, -3.9656203e-05, 5.913086e-05]",22050,"[-416.513579, 141.730557, 3.42186958, 22.13704..."
3,clip_0000fb6fe.wav,"[0.00039818, 0.00134888, 0.00260451]",22050,"[-445.238752, 139.523346, -66.4656067, 68.1054..."
4,clip_0001d1559.wav,"[-0.00214653, -0.00286958, -0.00289717]",22050,"[-372.486641, 69.4742626, -18.3678007, 38.2457..."


In [0]:
X_test = np.array(test_df.features.tolist())

In [0]:
X_test.shape

(158538, 40)

In [0]:
pred = model.predict(X_test.reshape(X_test.shape[0], 8, 5, 1))

In [0]:
y_pred = pd.Series(list(pred)).map(lambda x: np.argmax(x)).map(lambda x: lb.inverse_transform([x]))

In [0]:
my_results = pd.DataFrame([test_df.names, y_pred.str.get(0)]).T
my_results.columns = ['fname', 'label']
my_results.head()

,fname,label
0,clip_000044442.wav,right
1,clip_0000adecb.wav,down
2,clip_0000d4322.wav,on
3,clip_0000fb6fe.wav,right
4,clip_0001d1559.wav,yes


In [0]:
my_results.to_csv('sample_submission.csv')